## <center><font size="5" color='black'> ------------- Section 03 - Pipelines ------------- </font>

In [2]:
import utils as u
import boto3

### <font size="2" color='red'>Pull up artifacts that are provided by development team/reject the model</font>

In [3]:
"""
# What : In previous step, we (as a development team) have completed development and testing of notebook. As a output, the development
         team has produced below artificats for deploymet team
         1. preProcessRawData.py : Script to pre-process new incoming data in production before being provided to our model. Our model is trained on
            pre-processed data, hence whenever we get a new data in production we must pre-process it before givig it to model for better predictions.
            
         2. model.gz : This is our trained model that the development team has requested to be deployed in production.
         
         3. eval_metrics.csv : This is the evaluation metrics which is also provided by development team. Based on eval metrics, deployment team can
            make decision (using code or manual) whether to approve the model for production deployment or reject it.

# Why : We have purposely created a separate notebook for deployment team. This is for better understand that the previous notebook was developed with
        developer mindset. This notebook is now for deployer mindset. The deployment team is more focussed on validating model for prod deployment, 
        builing pipelines, making decision on data drift, model drift etc. 

# Anything Else : No

"""

# declare some root folder variables
my_bucket_name= '01-attempts-406016308324'

prefix_s3_deployer = r'pipelines/use_case_ageVsSalary/deployerRegion'

root_s3_deployer = r's3://01-attempts-406016308324/pipelines/use_case_ageVsSalary/deployerRegion'
root_local_folder = '/root/git/sandeep-gaadhe-git/sagemaker-implementations/pipelines'

local_data_folder = fr'{root_local_folder}/data'
local_deploy_folder = fr'{root_local_folder}/deployment/'

deployment_folder = fr'{root_s3_deployer}/deployment_20240823123423/'

bucket_name, prefix, files = u.get_all_files_from_s3_url(s3_url = deployment_folder, local_url = local_deploy_folder)

downloaded /root/git/sandeep-gaadhe-git/sagemaker-implementations/pipelines/deployment/eval_metrics.csv
downloaded /root/git/sandeep-gaadhe-git/sagemaker-implementations/pipelines/deployment/model.tar.gz
downloaded /root/git/sandeep-gaadhe-git/sagemaker-implementations/pipelines/deployment/preprocessRawData.py


In [ ]:
# print(fr'{deployment_folder}model.tar.gz')
# # print(bucket_name, prefix, files)
# for file in files:
#     if file == 'model.tar.gz':
#         print(fr's3://{bucket_name}/{prefix}{file}')
#     else:
#         continue

### <font size="2" color='red'>Register the model with RMSE as metric to approve/reject the model</font>

In [4]:
pending : get eval metric and create a decision code to either approve or reject the model.
df_eval_metrics = pd.read_csv(r'./deployment/', header=1, index_col="ID")

SyntaxError: invalid syntax (211756969.py, line 1)

In [ ]:
# getting the linear learner image according to the region
from sagemaker.image_uris import retrieve

base_image_linear_learner = retrieve("linear-learner", boto3.Session().region_name, version="1")
print(f'base_image_linear_learner : {base_image_linear_learner}')

In [ ]:
from sagemaker.model import Model
from sagemaker.predictor import Predictor
import sagemaker

# stored_model_artifact = fr'./deployment/model.tar.gz'

stored_model_artifact = fr'{deployment_folder}model.tar.gz'
ldn_time = u.get_london_time()
role = sagemaker.get_execution_role()
model_name="AgeVsSalary-model-" + ldn_time

# this does not create an entry under SageMaker >> Inferences >> Models
# or SageMaker Studio >> Models >> Model Registry
stored_model = Model(
                        image_uri=base_image_linear_learner,
                        model_data=stored_model_artifact,
                        role=role,
                        name=model_name,
                        predictor_cls=Predictor
                        )

# todo
stored_model.create(stored_model.name)
print(f'Created model {stored_model.name}')

In [ ]:
# model.register is used to maintan model versioning. The method makes use of model already created using model.create method.

# In Amazon SageMaker, when you register a model using the register() method, SageMaker automatically assigns a version number to the model within the specified Model Package Group. Unfortunately, you cannot explicitly set a custom version number when registering the model. The version numbers are sequential and managed by SageMaker.

# However, you can track and manage your versions effectively using tags, metadata, or descriptions. If you need to refer to a specific version with a custom identifier, you can include that information in the model's description or tags when registering the model.

# if the below code is executed twice than SageMaker will create Version 1 and Version 2 under SageMaker Studio >> Models >> Model Registry

model_package_group_name = "model-package-group-ageVsSalary"
model_approval_status = "PendingManualApproval"

register_args = stored_model.register(
                                        content_types=["text/csv"],
                                        response_types=["text/csv"],
                                        inference_instances=["ml.t2.medium", "ml.m5.xlarge"],
                                        transform_instances=["ml.m5.xlarge"],
                                        model_package_group_name=model_package_group_name,
                                        # model_package_description='Model AgeVsSalary - LinearLearner - Version 1.0', 
                                        approval_status=model_approval_status
                                        )

### <font size="2" color='red'>Simultaneously, create a sample pipeline for deploying the model, if approved</font>

### <font size="2" color='red'>Execute pipeline to deploy the model</font>

### <font size="2" color='red'>Upload new unseen data and generate inference by using model deployed thru pipeline</font>

## <center><font size="5" color='black'> ------------- Section 04 - DefaultModelMonitor ------------- </font>

## <font size="3" color='blue'>DefaultModelMonitor : Create code to monitor data quality and accept/reject batch input file based on constraints</font>

### <font size="2" color='blue'>Code for monitor data quality config and job using DefaultModelMonitor</font>

### <font size="2" color='blue'>Get a batch input file with outlier that the monitor data quality step should REJECT</font>

### <font size="2" color='blue'>Get a batch input file with outlier that the monitor data quality step should ACCEPT</font>

## <center><font size="5" color='black'> ------------- Section 05 - ModelMonitor ------------- </font>

## <font size="3" color='green'>ModelMonitor : Create code to monitor data quality and accept/reject batch input file based on constraints</font>

### <font size="2" color='green'>Code for monitor data quality config and job using ModelMonitor</font>

### <font size="2" color='green'>Get a batch input file with outlier that the monitor data quality step should REJECT</font>

### <font size="2" color='green'>Get a batch input file with outlier that the monitor data quality step should ACCEPT</font>

# <font size="4" color='black'>End Of Notebook</font>